In [13]:
import requests
import time

In [1]:
class Stack():
    def __init__(self):
        self.stack = []
    def push(self, value):
        self.stack.append(value)
    def pop(self):
        if self.size() > 0:
            return self.stack.pop()
        else:
            return None
    def size(self):
        return len(self.stack)

In [2]:
class Graph:
    """Represent a graph as a dictionary of vertices mapping labels to edges."""
    def __init__(self):
        self.vertices = {}
    def add_vertex(self, vertex):
        """
        Add a vertex to the graph.
        """
        self.vertices[vertex] = {}

    def add_direcs(self, vertex, exits):
        """
        Add a directed edge to the graph.
        """
        for i in exits:
            self.vertices[vertex].update({i:'?'})

    def get_neighbors(self, vertex_id):
        return self.vertices[vertex_id]

In [62]:
# Utils
inverseMap = {'n':'s', 's':'n', 'e':'w', 'w':'e'}

In [61]:
setupTable = {
    "url": 'https://lambda-treasure-hunt.herokuapp.com/api/',
    "token": "d2679616e63cea70c89ff019addc2677a27c9388",
    "init": 'adv/init/',
    "move": 'adv/move/',
}

header={'Authorization': 'Token ' + setupTable['token']}
init = setupTable['url'] + setupTable['init']
move = setupTable['url'] + setupTable['move']

In [149]:
class Player:
    
    def __init__(self):
        self.response = requests.get(url=init, headers=header)
        self.playersMap = {}
        self.coolDown = None
        self.currentRoomID = self.getCurrentRoomId()
        self.previousRoomID = None
        self.lastMoveDirection = None
        
        
    def refresh(self):
        self.response = requests.get(url=init, headers=header)
        return self.response.json()
    
    def getRoomInfo(self):
        if self.response.status_code == 200:
            return self.response.json()
        else:
            print(f'Unsuccessful: {self.response.status_code}')
            
    def getCurrentRoomId(self):
        return self.getRoomInfo()['room_id']
            
    def writeNewRoomToMap(self):
        exits = self.getRoomInfo()['exits']
        
        if self.currentRoomID not in self.playersMap:
            self.playersMap[self.currentRoomID] = {key: '?' for key in exits}
            
    def writeMoveDirectionsForCurrentAndPreviousRoom(self, previousRoomID, currentRoomID, direction):
        self.playersMap[previousRoomID][direction] = self.currentRoomID
        self.playersMap[currentRoomID][inverseMap[direction]] = self.previousRoomID
    
    def moveRoom(self, direction):
        if self.currentRoomID in self.playersMap and self.playersMap[self.currentRoomID][direction] !='?':
            print('Wise Explore')
            
            self.previousRoomID = self.currentRoomID
            
            nextRoom = self.playersMap[self.currentRoomID][direction]
            postRequestBody = {"direction":direction, "next_room_id":str(nextRoom)}
            self.response = requests.post(url=move, headers=header, json=postRequestBody)

            self.currentRoomID = self.getCurrentRoomId()
            self.coolDown = self.response.json()['cooldown']
            self.lastMoveDirection = direction
        
        else:
            print('Normal Explore')
            self.previousRoomID = self.currentRoomID
            
            postRequestBody = {"direction":direction}
            self.response = requests.post(url=move, headers=header, json=postRequestBody)
            
            self.currentRoomID = self.getCurrentRoomId()
            
            if self.response.status_code == 200:
                self.coolDown = self.response.json()['cooldown']
                self.lastMoveDirection = direction
                
                self.writeNewRoomToMap()
                self.writeMoveDirectionsForCurrentAndPreviousRoom(self.previousRoomID, self.currentRoomID, direction)
            else:
                print("The move was not successful, check coolDown attribute or current room exits")
            
        return self.getRoomInfo()
                
    
    def findUnexploredRoom(self):
        for direction in self.playersMap[self.currentRoomID]:
            if self.playersMap[self.currentRoomID][direction] == '?':
                return direction
        return direction
        
        

In [150]:
player = Player()

In [97]:
player.getRoomInfo()

{'room_id': 73,
 'title': 'A misty room',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'coordinates': '(59,64)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': [],
 'items': [],
 'exits': ['e'],
 'cooldown': 1.0,
 'errors': [],
 'messages': []}

In [98]:
player.currentRoomID

73

In [100]:
player.writeNewRoomToMap()

In [103]:
player.playersMap

{73: {'e': 63}, 63: {'n': '?', 's': '?', 'w': 73}}

In [102]:
player.moveRoom('e')

Normal Explore


{'room_id': 63,
 'title': 'A misty room',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'coordinates': '(60,64)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': [],
 'items': [],
 'exits': ['n', 's', 'w'],
 'cooldown': 15.0,
 'errors': [],
 'messages': ['You have walked east.']}

In [104]:
player.findUnexploredRoom()

n
s
w


# Main Function

In [151]:
def exploreWorld(player):
    
    roomStack = Stack()
    moveStack = Stack()
    
    roomStack.push(player.currentRoomID)
    
    roomSet = set()
    
    player.writeNewRoomToMap()
    roomSet.add(player.currentRoomID)
    
    while roomStack.size() > 0:
        
        direction = player.findUnexploredRoom()
        print("Direction", direction)
        
        if player.playersMap[player.currentRoomID][direction] == '?':
            
            player.moveRoom(direction)
            moveStack.push(direction)
            
            print("Our player has left room"+str(player.previousRoomID)+"and has entered room"+str(player.currentRoomID))
            
            roomStack.push(player.currentRoomID)
            
            if player.currentRoomID not in roomSet:
                roomSet.add(player.currentRoomID)
            
            if player.coolDown:
                time.sleep(player.coolDown)
        else:
            print("roomStack:", roomStack.stack)
            roomStack.pop()
            lastMove = moveStack.pop()
            
            if roomStack.size() > 0:
                player.previousRoomID = player.currentRoomID
                player.move([inverseMap[lastMove]])
                player.lastMoveDirection = moveStack[-1]
            if player.coolDown: 
                time.sleep(player.coolDown)
                
    return "You have traversed the world!"
        

In [152]:
exploreWorld(player)

Direction s
Normal Explore
Our player has left room72and has entered room63
Direction s
Normal Explore
Our player has left room63and has entered room20
Direction s
Normal Explore
Our player has left room20and has entered room19
Direction s
Normal Explore
Our player has left room19and has entered room10
Direction s
Normal Explore
Our player has left room10and has entered room0
Direction s
Normal Explore
Our player has left room0and has entered room2
Direction s
Normal Explore
Our player has left room2and has entered room6
Direction w
Normal Explore
Our player has left room6and has entered room7
Direction n
Normal Explore
Our player has left room7and has entered room8


KeyboardInterrupt: 

In [153]:
player.playersMap

{72: {'s': 63, 'w': '?'},
 63: {'n': 72, 's': 20, 'w': '?'},
 20: {'n': 63, 's': 19, 'e': '?', 'w': '?'},
 19: {'n': 20, 's': 10, 'w': '?'},
 10: {'n': 19, 's': 0, 'w': '?'},
 0: {'n': 10, 's': 2, 'e': '?', 'w': '?'},
 2: {'n': 0, 's': 6, 'e': '?'},
 6: {'n': 2, 'w': 7},
 7: {'n': 8, 'e': 6, 'w': '?'},
 8: {'s': 7, 'w': '?'}}